In [1]:
import torch
import numpy as np

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [7]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.6515, -2.2283, -0.4893],
        [ 0.4534,  0.2252, -0.2699]], requires_grad=True)
tensor([-1.2956, -0.2743], requires_grad=True)


In [8]:
def model(x):
    return x @ w.t() + b

# Generate predictions
preds = model(inputs)
print(preds)

# Compare with targets
print(targets)

tensor([[-219.1948,   36.3047],
        [-287.9928,   43.5263],
        [-384.9528,   53.6920],
        [-181.6732,   45.6677],
        [-294.4221,   33.7338]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [9]:
diff = (preds - targets)
diff = torch.sum(diff**2)/diff.numel()
diff


tensor(68335.7109, grad_fn=<DivBackward0>)

In [10]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff **2) / diff.numel()
    
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(68335.7109, grad_fn=<DivBackward0>)


In [16]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [14]:
print(w)
print(w.grad)

tensor([[-0.6515, -2.2283, -0.4893],
        [ 0.4534,  0.2252, -0.2699]], requires_grad=True)
tensor([[-29141.6504, -33069.9141, -20006.7266],
        [ -3917.7852,  -5151.0781,  -3075.0020]])


In [15]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [28]:
preds = model(inputs)
loss = mse(preds, targets)
loss.backward()
print(loss)

tensor(21922.5820, grad_fn=<DivBackward0>)


In [26]:
print(w.grad)
print(b.grad)

tensor([[-19353.8867, -22533.3516, -13509.1699],
        [ -2479.2571,  -3593.7417,  -2116.3911]])
tensor([-233.6977,  -32.3220])


In [27]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [50]:
# Train for 100 epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.4991, grad_fn=<DivBackward0>)


In [53]:
preds

tensor([[ 56.9496,  70.2631],
        [ 82.3426, 100.6990],
        [118.7270, 132.9661],
        [ 21.1084,  37.0210],
        [101.8869, 119.1304]], grad_fn=<AddBackward0>)

In [54]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [73]:
import torch.nn as nn
from torch.utils.data import TensorDataset, dataloader
# Import nn.functional
import torch.nn.functional as F

In [55]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


In [67]:
# Define dataset
train_ds = TensorDataset(inputs, targets)

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [72]:
# Define model
model = nn.Linear(3, 2)
# Parameters
list(model.parameters())


[Parameter containing:
 tensor([[ 0.3474,  0.4904, -0.3210],
         [-0.1800,  0.1016,  0.3423]], requires_grad=True),
 Parameter containing:
 tensor([-0.5332, -0.4836], requires_grad=True)]

In [75]:
preds = model(inputs)
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(4212.1904, grad_fn=<MseLossBackward0>)


In [78]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [81]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 100 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [83]:
fit(1000, model, loss_fn, opt, train_dl)

Epoch [100/1000], Loss: 1.1520
Epoch [200/1000], Loss: 1.2984
Epoch [300/1000], Loss: 1.0631
Epoch [400/1000], Loss: 0.8564
Epoch [500/1000], Loss: 0.8808
Epoch [600/1000], Loss: 1.0536
Epoch [700/1000], Loss: 1.3666
Epoch [800/1000], Loss: 1.5112
Epoch [900/1000], Loss: 0.8889
Epoch [1000/1000], Loss: 0.7825


In [84]:
preds = model(inputs)
preds

tensor([[ 56.8361,  70.3511],
        [ 82.0599, 100.6389],
        [118.1569, 132.8607],
        [ 20.8020,  37.8041],
        [101.8244, 118.5347],
        [ 55.5939,  69.2767],
        [ 81.9213, 100.7384],
        [118.4593, 133.4705],
        [ 22.0442,  38.8785],
        [102.9280, 119.7086],
        [ 56.6975,  70.4507],
        [ 80.8177,  99.5645],
        [118.2955, 132.7612],
        [ 19.6984,  36.6302],
        [103.0666, 119.6091]], grad_fn=<AddmmBackward0>)

In [85]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])